In [1]:
#hide
# !pip install -r ../requirements.txt 

# Introducing fastpages with hvplot
> An easy to use blogging platform with extra features for <a href="https://jupyter.org/">Jupyter Notebooks</a>.

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: Thomas PEDOT
- image: images/diagram.png
- categories: [fastpages, jupyter]


In [2]:
#hide
from influxdb_client import InfluxDBClient

import pandas as pd
import time
import requests
import hvplot.pandas
from decouple import config
from urllib.parse import urlparse
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from snippets import *
TOKEN = config("TOKEN")
ORG = config("ORG")
INFLUXDB_URL = config("INFLUXDB_URL")


In [3]:
#hide
client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)

client.health()

{'checks': [],
 'commit': '4e7a59bb9a',
 'message': 'ready for queries and writes',
 'name': 'influxdb',
 'status': 'pass',
 'version': '2.0.4'}

In [9]:
# https://www.influxdata.com/blog/getting-started-with-influxdb-and-pandas/
query_api = client.query_api()
query = '''from(bucket: "energie")
  |> range(start: -7d)
  |> filter(fn: (r) => r["_measurement"] == "solarmanpv")
  |> filter(fn: (r) => r["_field"] == "Puissance de l'onduleur à stockage d'énergie")
  |> aggregateWindow(every: 1d, fn: mean)'''


result = client.query_api().query_data_frame(org=ORG, query=query)
result.columns = ['result', 'table', 'start', 'stop', 'time', 'value', 'field',
       'measurement', 'host', 'topic']
result.head()

,result,table,start,stop,time,value,field,measurement,host,topic
0,_result,0,2021-04-03 09:05:08.917741+00:00,2021-04-10 09:05:08.917741+00:00,2021-04-04 00:00:00+00:00,695.048263,Puissance de l'onduleur à stockage d'énergie,solarmanpv,telegraf1,SolarmanPV/realTimeDataBattery
1,_result,0,2021-04-03 09:05:08.917741+00:00,2021-04-10 09:05:08.917741+00:00,2021-04-05 00:00:00+00:00,596.041667,Puissance de l'onduleur à stockage d'énergie,solarmanpv,telegraf1,SolarmanPV/realTimeDataBattery
2,_result,0,2021-04-03 09:05:08.917741+00:00,2021-04-10 09:05:08.917741+00:00,2021-04-06 00:00:00+00:00,691.384358,Puissance de l'onduleur à stockage d'énergie,solarmanpv,telegraf1,SolarmanPV/realTimeDataBattery
3,_result,0,2021-04-03 09:05:08.917741+00:00,2021-04-10 09:05:08.917741+00:00,2021-04-07 00:00:00+00:00,526.028513,Puissance de l'onduleur à stockage d'énergie,solarmanpv,telegraf1,SolarmanPV/realTimeDataBattery
4,_result,0,2021-04-03 09:05:08.917741+00:00,2021-04-10 09:05:08.917741+00:00,2021-04-08 00:00:00+00:00,774.879121,Puissance de l'onduleur à stockage d'énergie,solarmanpv,telegraf1,SolarmanPV/realTimeDataBattery


In [15]:
#hide 
# https://www.influxdata.com/blog/streaming-time-series-with-jupyter-and-influxdb/
energie_hvplot = result.hvplot.bar(x="time", y="value", rot=45)
energie_hvplot

:Bars   [time]   (value)

In [16]:
#hide

filename = '../_includes/energie_hvplot.html'

    
export_plot_fastpages(energie_hvplot, filename)